In [1]:
import os, sys
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon

In [2]:
# set paths and directory name
geojson_dir = "/path/to/ports_project/geojson_out/"
geo_base_dir = "/path/to/ports_project/geo_data/"
port = 'directory_name'
road_file = port+"_TA.shp"
gpath = [geo_base_dir,port,"/",road_file]
geo_data_path = ''.join(gpath)
geo_data = gpd.read_file(geo_data_path)

In [3]:
# set paths
congestion_base_dir = "/path/to/ports_project/traffic_analytics/"
months = ["jan","feb","mar","apr","may","jun","jul","aug","sep","oct","nov"]
congestion_file_suffix = "_CONGESTION.csv"

In [4]:
# join the congestion file to the NavStreets shapefie with to and from LINK-IDs
for month in months:
    cpath = [congestion_base_dir,port,"/",month,"_",port,congestion_file_suffix]
    congestion_file_path = "".join(cpath)
    congestion_data = pd.read_csv(congestion_file_path)
    
    # set negative values to 0
    congestion_data.loc[congestion_data['CONGESTION-WEIGHTED'] < 0, 'CONGESTION-WEIGHTED'] = 0
    
    # group by road segment or linkId
    congestion_sum = congestion_data.groupby('LINK-DIR')['CONGESTION-WEIGHTED'].mean().reset_index()
    
    # create a common field name for merging
    geo_data["LINK-DIR"] = geo_data["LINK_DIR"]

    # merge congestion scores to streets
    congestion_geo = geo_data.merge(congestion_sum, on='LINK-DIR')
    
    # remove duplicates
    congestion_geo = congestion_geo.drop_duplicates()
    
    # write geojson summary file
    gjpath = [geojson_dir,port,"/"]
    geojson_path = "".join(gjpath)
    if not os.path.exists(geojson_path):
        os.makedirs(geojson_path)
    fpath = [geojson_path,month,"_",port,"_congestion_TA.geojson"]
    file = "".join(fpath)
    congestion_geo.to_file(file, driver="GeoJSON")